<a href="https://colab.research.google.com/github/JuanjoRestrepo/Qatar-2022/blob/main/WorldCup_Qatar2022_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle
import time
import numpy as np

# **1. Recolección de Datos**

In [2]:
#all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
all_tables = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

## **1.1 Obteniendo los grupos**

#### Como son 8 grupos (A -> H) y en el link las tablas de estos están ordenados cada 7 posiciones (desde la 12) haremos lo siguiente para obtener la tabla de cada grupo:
- 11 -> 7*8 + 12 = 67

In [3]:
all_tables[12]
all_tables[19]
all_tables[26]
all_tables[61]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ghana,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Uruguay,0,0,0,0,0,0,0,0,NaN
3,4,South Korea,0,0,0,0,0,0,0,0,NaN


Como vemos el último grupo (H) está en la posición 61, por lo cual haremos un recorrido hasta ese rango que calculamos anteriorment y haremos unas modificaciones al formato de la tabla original que nos presenta el dataset.

Reemplazaremos el nombre de la columna 'Teamvte' por solo 'Team' pero hay una excepción en el primer grupo (A) y es que tiene un formato de 'Team.mw' en su columna de equipos, por lo cual haremos lo siguiente

In [4]:
all_tables[12].columns[1]

'Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:"[ "}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:" ]"}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte'

Asignamos el valor numerico de las tablas a su grupo correspondiente

In [5]:
for letter, i in zip(alphabet, range(12, 68, 7)):
  print(letter, i)

A 12
B 19
C 26
D 33
E 40
F 47
G 54
H 61


In [6]:
dict_tables = {}
for letter, i in zip(alphabet, range(12, 68, 7)):
  df = all_tables[i]
  df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
  df.pop('Qualification')
  dict_tables[f'Group {letter}'] = df

In [7]:
dict_tables.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [8]:
dict_tables['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


## **1.2 Exportamos nuestro Diccionario**

#### Abrimos un archivo para colocar el diccionario de las tablas en un archivo llamado 'output'

In [9]:
with open('dict_table', 'wb') as output:
  pickle.dump(dict_tables, output)

# **2. Extraemos la data de todos los mundiales desde 1930 hasta 2018 y de los partidos del 2022**

## **2.1 Data Histórica Partidos 1930 a 2018 y la Data de Qatar 2022**

#### Haremos el Webscrapping para obtener los partidos de **1930** hasta **2018** con 'requests' y bs4

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Extraemos la data de todos los mundiales desde 1930 hasta 2018
# Haremos el Webscrapping para obtener estos datos con 'requests' y bs4
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

def get_matches(year):
  if year == '2022':
    web =  f'https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
  else:
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

  response = requests.get(web)
  content = response.text #contenido html de la pagina
  soup = BeautifulSoup(content, 'lxml')

  matches = soup.find_all('div', class_='footballbox')

  home = []
  score = []
  away = []

  for game in matches:
      home.append(game.find('th', class_='fhome').get_text())
      score.append(game.find('th', class_='fscore').get_text())
      away.append(game.find('th', class_='faway').get_text())

  dict_football = {'home': home,
                  'score': score,
                  'away': away}

  df_football = pd.DataFrame(dict_football)
  df_football['year'] = year
  return df_football

# Data Historica de todos los mundiales realizados
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)

# Data del mundial Qatar 2022
df_fixture = get_matches('2022')
df_fixture = df_fixture
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)

#### Usando Chromedriver hacemos el WebScrapping para obtener un DataFrame de todos los mundiales hasta el 2018

#### Nota: el webscrapping se realizó tanto en Google en Colab, como de forma local en un script a parte llamado "selenium-world-cup.py"

In [11]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [770 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,570 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,354 kB]
Hit:10 https://ppa.l

## **2.2 Web Scrapping de la Data Faltante**

#### Obtenemos los datos faltantes a partir del WebScrapping con Chrome Webdriver

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

#path = 'C:\Users\Juan Jose Restrepo\Desktop\WC 2022\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

def getMissingData(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    print(f'\nGetting the Matches of WC {year}')

    # NODO PADRE: CUBRE LOCAL Y VISITANTE: <tr itemprop="name"> <th class="fhome" itemprop="homeTeam" itemscope="" itemtype="http://schema.org/SportsTeam"><span itemprop="name"><a href="/wiki/Italy_national_football_team" title="Italy national football team">Italy</a><span class="flagicon">&nbsp;<span class="mw-image-border" typeof="mw:File"><span><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/23px-Flag_of_Italy.svg.png" decoding="async" width="23" height="15" class="mw-file-element" srcset="//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/35px-Flag_of_Italy.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/45px-Flag_of_Italy.svg.png 2x" data-file-width="1500" data-file-height="1000"></span></span></span></span></th><th class="fscore">0–0</th><th class="faway" itemprop="awayTeam" itemscope="" itemtype="http://schema.org/SportsTeam"><span itemprop="name"><span style="white-space:nowrap"><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/1/12/Flag_of_Poland.svg/23px-Flag_of_Poland.svg.png" decoding="async" width="23" height="14" class="mw-file-element" srcset="//upload.wikimedia.org/wikipedia/en/thumb/1/12/Flag_of_Poland.svg/35px-Flag_of_Poland.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/1/12/Flag_of_Poland.svg/46px-Flag_of_Poland.svg.png 2x" data-file-width="1280" data-file-height="800"></span></span>&nbsp;</span><a href="/wiki/Poland_national_football_team" title="Poland national football team">Poland</a></span></span></th></tr>
    # <th class="fhome" itemprop="homeTeam" itemscope="" itemtype="http://schema.org/SportsTeam"><span itemprop="name"><a href="/wiki/Italy_national_football_team" title="Italy national football team">Italy</a><span class="flagicon">&nbsp;<span class="mw-image-border" typeof="mw:File"><span><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/23px-Flag_of_Italy.svg.png" decoding="async" width="23" height="15" class="mw-file-element" srcset="//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/35px-Flag_of_Italy.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/45px-Flag_of_Italy.svg.png 2x" data-file-width="1500" data-file-height="1000"></span></span></span></span></th>

    #//th[@class="fhome"]/..

    # Find all rows containing match information
    # obtenemos los partidos en la pagina web
    driver.get(web)
    matches = driver.find_elements(by='xpath', value='//th[@class="fhome"]/..')

    # guardamos los datos de los partidos en las listas
    home = []
    score = []
    away = []

    # Recorremos los partidos guardados para separarlos en local, visitante y resultado
    for match in matches:
        home.append(match.find_element(by='xpath', value='./th[1]').text)
        score.append(match.find_element(by='xpath', value='./th[2]').text)
        away.append(match.find_element(by='xpath', value='./th[3]').text)

    # Creamos un DataFrame a partir de las listas
    data = {'Home': home, 'Score': score, 'Away': away}
    df_football = pd.DataFrame(data)
    df_football['year'] = year
    time.sleep(2)

    return df_football



years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

# Guardamos todos los df de los mundiales en una lista
fifa = [getMissingData(year) for year in years]
# Close the WebDriver
driver.quit()

# Juntamos todos los df en uno solo
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_missing_data.csv', index=False)

print('Web Scraping Done!')


Getting the Matches of WC 1930

Getting the Matches of WC 1934

Getting the Matches of WC 1938

Getting the Matches of WC 1950

Getting the Matches of WC 1954

Getting the Matches of WC 1958

Getting the Matches of WC 1962

Getting the Matches of WC 1966

Getting the Matches of WC 1970

Getting the Matches of WC 1974

Getting the Matches of WC 1978

Getting the Matches of WC 1982

Getting the Matches of WC 1986

Getting the Matches of WC 1990

Getting the Matches of WC 1994

Getting the Matches of WC 1998

Getting the Matches of WC 2002

Getting the Matches of WC 2006

Getting the Matches of WC 2010

Getting the Matches of WC 2014

Getting the Matches of WC 2018
Web Scraping Done!


### **NOTA: AMBOS METODOS DE WEBSCRAPPING FUNCIONAN**

# **3. Limpieza de los Datos**

#### Cargamos los dataframes historico y el de Qatar 2022

In [13]:
df_data_historica = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_data_faltante = pd.read_csv('fifa_worldcup_missing_data.csv')

df_data_historica

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
860,Russia,2–2 (a.e.t.),Croatia,2018
861,France,1–0,Belgium,2018
862,Croatia,2–1 (a.e.t.),England,2018
863,Belgium,2–0,England,2018


In [14]:
df_data_faltante

,Home,Score,Away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
860,Russia,2–2 (a.e.t.),Croatia,2018
861,France,1–0,Belgium,2018
862,Croatia,2–1 (a.e.t.),England,2018
863,Belgium,2–0,England,2018


### Dado que la data de ambos DFs es igual, no hay necesidad de concatenarlos, pues no hay datos faltantes. El webscrapping fue realizado correctamente

## **3.1 Limpiando df_fixture**

### Cargamos la Data de los partidos por jugar en Qatar 2022

In [15]:
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


Posteriormente, separaremos las columnas de ***'Home'*** y ***'Away'***

In [16]:
df_fixture['home']

0               Qatar 
1             Senegal 
2               Qatar 
3         Netherlands 
4             Ecuador 
            ...       
59    Winners Match 51
60    Winners Match 57
61    Winners Match 59
62     Losers Match 61
63    Winners Match 61
Name: home, Length: 64, dtype: object

Si vemos en lo anterior, podemos apreciar que hay datos en blanco, los cuales son espacios de caracteres. Por ejemplo en 'Qatar', tenemos
'__Qatar'

Para corregir esto, utilizaremos el método str.strip()

In [17]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

## **3.2 Limpiando df_data_historica**

#### Chequeamos que no haya data faltante o datos nulos en el **DF Data Historica**

In [18]:
df_data_historica[df_data_historica['home'].isnull()]

,home,score,away,year


In [19]:
df_data_historica[df_data_historica['score'].isnull()]

,home,score,away,year


In [20]:
df_data_historica[df_data_historica['away'].isnull()]

,home,score,away,year


In [21]:
df_data_historica[df_data_historica['year'].isnull()]

,home,score,away,year


#### Chequeamos que no haya data faltante o datos nulos en el **DF Data Faltante**

In [22]:
df_data_faltante[df_data_faltante['Home'].isnull()]

,Home,Score,Away,year


In [23]:
df_data_faltante[df_data_faltante['Score'].isnull()]

,Home,Score,Away,year


In [24]:
df_data_faltante[df_data_faltante['Away'].isnull()]

,Home,Score,Away,year


In [25]:
df_data_faltante[df_data_faltante['year'].isnull()]

,Home,Score,Away,year


### Eliminamos datos duplicados en la Data Histórica y los ordenamos por año de menor a mayor

In [26]:
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values('year', inplace=True)
df_data_historica

,home,score,away,year
0,France,4–1,Mexico,1930
9,Romania,3–1,Peru,1930
10,Uruguay,1–0,Peru,1930
11,Uruguay,4–0,Romania,1930
12,United States,3–0,Belgium,1930
...,...,...,...,...
804,Uruguay,1–0,Saudi Arabia,2018
805,Uruguay,3–0,Russia,2018
806,Saudi Arabia,2–1,Egypt,2018
807,Morocco,0–1,Iran,2018


Dado que hay un partido que tiene **Walk Over**, es decir, uno que no se jugo y termino en 3 a 0.

Esto no es beneficioso dado que esta data no es una data real, sino que es arbitraria que puede afectar nuestra prediccion para el ganador del 2022

Dicho partido es el siguiente:

In [27]:
df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')]

,home,score,away,year
37,Sweden,w/o,Austria,1938


Como vemos, este partido no tiene un score numerico, por lo que no nos aporta. Por lo cual, lo eliminaremos

In [28]:
index_eliminar = df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')].index

df_data_historica.drop(index = index_eliminar, inplace=True)

In [29]:
# Comprobamos que lo eliminamos correctamente
df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')]

,home,score,away,year


#### Corroboramos que todos los datos de la columna 'Score' sean de tipo numerico, por medio de Expresiones Regulares

In [30]:
df_data_historica['score'] = df_data_historica['score'].str.replace('\[^d–]', '', regex=True)
df_data_historica[df_data_historica['score'].str.contains('\[^d–]')]

,home,score,away,year


### La expresión '\[^d–]' nos reemplaza cualquier expresión que sea diferente a un valor entero por vacío ''

### Reemplazamos

In [ ]:
# Segunda forma de eliminar los datos no numericos
#df_data_historica['score'] = df_data_historica['score'].str.replace('(a.e.t.)', '', regex=True)
#df_data_historica[df_data_historica['score'].str.contains('(a.e.t.)')]

### Limpiamos los datos: home, away y score eliminando cualquier caracter en blanco que acompañe

In [31]:
df_data_historica['home'] = df_data_historica['home'].str.strip()
df_data_historica['away'] = df_data_historica['away'].str.strip()

### Separamos ***'HomeGoals'*** y ***'AwayGoals'*** en dos columnas y eliminamos la columna de Score

In [32]:
df_data_historica[['HomeGoals', 'AwayGoals']] = df_data_historica['score'].str.split('–', expand=True)
df_data_historica

,home,score,away,year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
9,Romania,3–1,Peru,1930,3,1
10,Uruguay,1–0,Peru,1930,1,0
11,Uruguay,4–0,Romania,1930,4,0
12,United States,3–0,Belgium,1930,3,0
...,...,...,...,...,...,...
804,Uruguay,1–0,Saudi Arabia,2018,1,0
805,Uruguay,3–0,Russia,2018,3,0
806,Saudi Arabia,2–1,Egypt,2018,2,1
807,Morocco,0–1,Iran,2018,0,1


In [33]:
df_data_historica.drop('score', axis=1, inplace=True)
df_data_historica

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
9,Romania,Peru,1930,3,1
10,Uruguay,Peru,1930,1,0
11,Uruguay,Romania,1930,4,0
12,United States,Belgium,1930,3,0
...,...,...,...,...,...
804,Uruguay,Saudi Arabia,2018,1,0
805,Uruguay,Russia,2018,3,0
806,Saudi Arabia,Egypt,2018,2,1
807,Morocco,Iran,2018,0,1


### Renombramos las columnas ***home***, ***away*** y ***year***

In [34]:
df_data_historica.rename(columns={'home':'HomeTeam', 'away':'AwayTeam',
                                  'year':'Year'}, inplace=True)
df_data_historica

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
9,Romania,Peru,1930,3,1
10,Uruguay,Peru,1930,1,0
11,Uruguay,Romania,1930,4,0
12,United States,Belgium,1930,3,0
...,...,...,...,...,...
804,Uruguay,Saudi Arabia,2018,1,0
805,Uruguay,Russia,2018,3,0
806,Saudi Arabia,Egypt,2018,2,1
807,Morocco,Iran,2018,0,1


In [39]:
# Check the data types of the DataFrame
print(df_data_historica.dtypes)

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals    object
AwayGoals    object
dtype: object


In [40]:
# Identify the column with the problematic data type
print(df_data_historica['AwayGoals'].unique())


['1' '0' '3' '2' '1 (a.e.t.)' '2 (a.e.t.)' '5 (a.e.t.)' '3 (a.e.t.)'
 '0 (a.e.t.)' '4 (a.e.t.)' '5' '4' '7' '0 (a.e.t./g.g.)' '1 (a.e.t./g.g.)'
 '2 (a.e.t./g.g.)']


In [41]:
# Convert the column to a numeric type, ignoring non-numeric values
df_data_historica['AwayGoals'] = pd.to_numeric(df_data_historica['AwayGoals'], errors='coerce')


In [42]:
# Check the data types again to confirm the change
print(df_data_historica.dtypes)

HomeTeam      object
AwayTeam      object
Year           int64
HomeGoals     object
AwayGoals    float64
dtype: object


In [43]:
# Identify the column with the problematic data type
print(df_data_historica['AwayGoals'].unique())

[ 1.  0.  3.  2. nan  5.  4.  7.]


## **Convertimos los datos a int**

In [48]:
# Replace NaN and infinity values with 0
df_data_historica['AwayGoals'] = df_data_historica['AwayGoals'].fillna(0).replace([np.inf, -np.inf], 0)

In [51]:
# Convert the column to integer data type
df_data_historica = df_data_historica.astype({'HomeGoals': int, 'AwayGoals': int})

In [52]:
print(df_data_historica.dtypes)

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int64
AwayGoals     int64
dtype: object


In [ ]:
copiaAwayGoals.set_index('AwayGoals').filter(like=' (a.e.t.)', axis=0)

""
AwayGoals
1 (a.e.t.)
1 (a.e.t.)
2 (a.e.t.)
1 (a.e.t.)
0 (a.e.t.)
...
1 (a.e.t.)
1 (a.e.t.)
1 (a.e.t.)
